In [1]:
import os
os.chdir("../")

from tqdm.notebook import tqdm, trange
from pyinstrument import Profiler
from joblib import Parallel, delayed, parallel_backend

import semiolog as slg

semiotic = slg.Cenematic("en_bnc",requested_cpu=4)

from collections import Counter, defaultdict
from functools import reduce
import operator
import time

Vocabulary will not be loaded from file.



In [2]:
%load_ext line_profiler

In [16]:
semiotic.vocab.build_new(vocab_size = 400, parallel = True, corpus_length = 20000)
new_freq = semiotic.vocab.freq
new_merges = semiotic.vocab.merges
semiotic.vocab.build(vocab_size = 400, parallel = True, corpus_length = 20000)

Computing in parallel
Normalize and jobs data...
... computed in 0.8924679756164551 secs.

Build alphabet...
... computed in 0.00046372413635253906 secs.

Alphabet Size: 78
Special Tokens Size: 5
Terms to compute: 317

Enter loop



Compute freq...
... computed in 0.29940295219421387 secs.

Vocabulary built
Computing in parallel
Normalize and jobs data...
... computed in 1.3269822597503662 secs.

Build alphabet...
... computed in 0.0007753372192382812 secs.

Alphabet Size: 78
Special Tokens Size: 5
Terms to compute: 317

Enter loop



Compute freq...
... computed in 0.30179309844970703 secs.

Vocabulary built


In [17]:
for n,o in zip(new_merges,semiotic.vocab.merges):
    if n!=o:
        print(n,"-",o)

c t 1432 - m ent 1432
m ent 1432 - c t 1432
p e 892 - n o 892
n o 892 - p e 892
u re 854 - oun d 854
oun d 854 - u re 854
wh en 674 - com p 674
com p 674 - wh en 674
the s 642 - wor k 642
wor k 642 - the s 642
i p 634 - as s 634
as s 634 - i p 634
e p 629 - im e 629
im e 624 - e p 624
ou s 621 - it ion 621
it ion 621 - ou s 621
i re 619 - a ve 619
a ve 619 - i re 619
v i 611 - w ay 611
w ay 611 - v i 611
p res 600 - p res 601
p re 568 - p re 569
c oun 567 - k now 567
k now 567 - c oun 567
d u 525 - be t 525
be t 525 - d u 525
it is 519 - am e 519
am e 519 - it is 519
an c 504 - ct ion 504
ct ion 504 - an c 504
d ay 501 - you r 501
you r 501 - d ay 501
in k 490 - and the 490
and the 490 - in k 490
it t 463 - h ow 463


In [41]:
for key in new_freq.keys():
    if new_freq.get(key,"-")!=semiotic.vocab.freq.get(key,"-"):
        print(key,new_freq.get(key,"-"),semiotic.vocab.freq.get(key,"-"))

s 7 5
e 6 4
ti 5 3
o 5 4
n 5 7
f 4 3
p 4 3
ers 4 3
c 3 2
r 3 5
d 3 5
i 2 6
and 2 -
ri 2 -
thersname 2 1
ni 1 -
j 1 -
iv 1 -
in 1 -
is 1 -
at 1 -


In [5]:
# clean
%lprun -f semiotic.vocab.build_new semiotic.vocab.build_new(vocab_size = 200, parallel = True, corpus_length = 2000)

Computing in parallel
Normalize and jobs data...
... computed in 0.4826979637145996 secs.

Build alphabet...
... computed in 0.014801025390625 secs.

Alphabet Size: 49
Special Tokens Size: 5
Terms to compute: 146

Enter loop



Compute freq...
... computed in 0.07721495628356934 secs.

Vocabulary built


Timer unit: 1e-06 s

Total time: 1.74738 s
File: /Users/Gianni/semiolog/semiolog/vocabulary.py
Function: build_new at line 433

Line #      Hits         Time  Per Hit   % Time  Line Contents
   433                                               def build_new(
   434                                                   self,
   435                                                   corpus = None,
   436                                                   vocab_size = None,
   437                                                   special_tokens = None,
   438                                                   save = False,
   439                                                   save_step = None,
   440                                                   progress_bar = True,
   441                                                   resume_merges = False,
   442                                                   parallel = False,
   443                                                   corpus_length 

In [19]:
def pre_process(corpus_chunk, normalizer):
    # Normalize
    chain_zip = normalizer(corpus_chunk)
    # Build list of pairs
    chain_zip = list(zip(chain_zip,chain_zip[1:]))
    # Create a lookup table of all the positions where a pair appears in a corpus
    pair_pos = defaultdict(set)
    for i,k in list(enumerate(chain_zip)):
        pair_pos[k].add(i)
    # From the previous lookup table, create another lookup table of the frequency of each pair (given by the size of the set of its positions)
    pair_len = Counter()
    for k,pos in pair_pos.items():
        pair_len[k] = len(pos)
    
    return (chain_zip, pair_pos, pair_len)

def process_best_pair(chain_zip, pair_pos, best_pair):
    chain_zip_len = len(chain_zip)
    pair_len_delta = Counter()

    for i in pair_pos[best_pair]:
        # Skip iteration if position corresponds to a modified set of positions during the iteration. This can happen if there is overlap of pairs, such as "000", where ("0","0") has itself as right pair. Note that, due to unordered implementation of sets, this entails a lack of systematicity in overlapping cases: "000" can be counted randomly as ("00","0") or ("0","00").
        # TODO: Investigate the cost of ordering sets. In which case, the following "if" condition might only be needed for right pairs.
        if chain_zip[i]!=best_pair:
            continue
        ## merge best pair with left unit
        left_pair_i = i-1
        while left_pair_i>=0 and chain_zip[left_pair_i] == None: # if left pair is within chain limits but empty (= None) because already merged previously, shift to the left
            left_pair_i -= 1
        if left_pair_i>-1: # proceed only if a left pair was found on the left
            # Remove from left pair positions, the current position (of the pair to be merged)
            left_pair = chain_zip[left_pair_i]
            # Skip update of left_pair position set if left_pair = best_pair, to avoid modification of iterating set. This can happen if there is overlap of pairs. No consequences on final result (right?) since right after the loop, the key corresponding to the best pair is deleted, and chain_zip is indeed updated so the problematic cases can be captured at the beginning of the loop.
            if left_pair != best_pair:
                left_pair_pos = pair_pos[left_pair]
                left_pair_pos.discard(left_pair_i)
            new_pair = (left_pair[0],"".join(best_pair)) # construct new left pair
            pair_pos[new_pair].add(left_pair_i) # add new pair (if non existing) and its position to the pair_pos lookup table
            # update the counts in the pair_len lookuptable
            pair_len_delta[left_pair] -= 1
            pair_len_delta[new_pair] += 1

            # update the list of pairs
            chain_zip[left_pair_i] = new_pair

        ## merge best pair with right unit.
        # Code is symmetric to left_pair but on the right. Comments are omitted
        right_pair_i = i+1
        while right_pair_i<chain_zip_len and chain_zip[right_pair_i] == None:
            right_pair_i += 1
        if right_pair_i<chain_zip_len:
            right_pair = chain_zip[right_pair_i]
            if right_pair != best_pair:
                right_pair_pos = pair_pos[right_pair]
                right_pair_pos.discard(right_pair_i)
            new_pair = ("".join(best_pair), right_pair[1])
            pair_pos[new_pair].add(right_pair_i)
            pair_len_delta[right_pair] -= 1
            pair_len_delta[new_pair] += 1

            chain_zip[right_pair_i] = new_pair

        # Empty best pair position in list of pairs
        chain_zip[i] = None

    # Remove best pair from lookuptables
    del pair_pos[best_pair]

    return (chain_zip, pair_pos, pair_len_delta)

def compute_freq(chain_zip):
    # TODO: add the last unit to the decoupling
    freq = [pair[0] for pair in chain_zip if pair != None]
    if chain_zip[-1]!=None: 
        freq.append(chain_zip[-1][-1])
    freq = Counter(freq)
    return freq

In [23]:
vocab_size = 2000
corpus_length = 2000

self = semiotic.vocab
corpus = None

special_tokens = None
save = False
save_step = None
truncate_best_size = None
progress_bar = True
resume_merges = False
parallel = False

if corpus == None:
    corpus = self.name

if vocab_size == None:
    vocab_size = self.config.size

if special_tokens == None:
    special_tokens = self.config.special_tokens

if corpus_length == None:
    corpus_length = self.corpus.train_len

saveQ = False

In [ ]:
def test(
    vocab_size = vocab_size,
    corpus_length = corpus_length,

    self = self,
    corpus = corpus,

    special_tokens = special_tokens,
    save = save,
    save_step = save_step,
    progress_bar = progress_bar,
    resume_merges = resume_merges,
    parallel = parallel,

    
):



In [25]:

chunksize = int(corpus_length/self.cpu_count)

corpus_chunks = ["".join(self.corpus.train[i*chunksize:i*chunksize+chunksize]) for i in range(0,self.cpu_count)]

with Parallel(n_jobs=self.cpu_count, require='sharedmem') as parallel_pool:
    print("Computing in parallel")
    print("Normalize and jobs data...")
    start = time.time()
    jobs_data = parallel_pool(delayed(pre_process)(chunk,self.normalizer.normalize) for chunk in corpus_chunks)

    pair_len_global = reduce(operator.add,[i[-1] for i in jobs_data])

    best_pair, best_pair_len = max(pair_len_global.items(), key=operator.itemgetter(1))

    merges = [" ".join(best_pair)]
    print(f"... computed in {time.time()-start} secs.\n")

    print("Build alphabet...")
    start = time.time()
    alphabet = Counter()
    for (l,r),v in pair_len_global.items():
        alphabet[l] += v
    # In extreme cases, right characters of pairs might not be left characters. If there are such chars, they're added with freq 1
    left_out_chars = {r for l,r in pair_len_global.keys()}-alphabet.keys()
    if len(left_out_chars)>0:
        print(f"Adding characters: {left_out_chars}")
        for char in left_out_chars:
            alphabet[char] += 1
    print(f"... computed in {time.time()-start} secs.\n")

    alpha_len = len(alphabet)
    special_tokens_len = 0 if special_tokens == None else len(special_tokens)
    
    print(f"Alphabet Size: {alpha_len}")
    print(f"Special Tokens Size: {special_tokens_len}")

    
    if vocab_size<0:
        voc_final_length = alpha_len + abs(vocab_size) + special_tokens_len
    else:
        voc_final_length = vocab_size

    delta_voc = voc_final_length - alpha_len - special_tokens_len

    print(f"Terms to compute: {delta_voc}\n")

    print("Enter loop")

    t = trange(delta_voc, disable = not progress_bar)
    for _ in t:
        t.set_description(f"Pair: {best_pair}, {best_pair_len}")
        t.refresh()

        jobs_data = parallel_pool(delayed(process_best_pair)(chain_zip, pair_pos, best_pair) for chain_zip, pair_pos, pair_len_delta in jobs_data)

        for chain_zip, pair_pos, pair_len_delta in jobs_data:
            pair_len_global.update(pair_len_delta)

        # Remove best_pair from pair_len
        del pair_len_global[best_pair]

        best_pair, best_pair_len = max(pair_len_global.items(), key=operator.itemgetter(1))

        merges.append(" ".join(best_pair))
        # print(f"... computed in {time.time()-start} secs.\n")

    print("Compute freq...")
    start = time.time()
    freqs = parallel_pool(delayed(compute_freq)(job_data[0]) for job_data in jobs_data)
    freq = reduce(operator.add, freqs)
    print(f"... computed in {time.time()-start} secs.\n")

Computing in parallel
Normalize and jobs data...
... computed in 0.11342406272888184 secs.

Build alphabet...
... computed in 0.0003790855407714844 secs.

Alphabet Size: 49
Special Tokens Size: 5
Terms to compute: 1946

Enter loop



Compute freq...
... computed in 0.0218658447265625 secs.



In [29]:
for key in new_freq.keys():
    if new_freq[key]!=semiotic.vocab.freq[key]:
        print(key,new_freq[key],freq[key],semiotic.vocab.freq[key])

and 510 510 457
to 443 431 469
s 432 441 413
in 417 417 376
ed 413 404 383
ing 413 405 389
the 379 379 339
of 378 366 347
as 325 315 302
is 309 310 287
al 283 283 270
er 276 270 290
it 267 267 254
es 267 277 272
at 256 256 272
on 255 248 256
m 254 255 247
an 252 253 232
or 245 245 237
t 244 242 235
that 230 230 212
i 229 229 248
c 224 219 200
‘ 224 224 210
y 220 220 223
ly 219 213 185
p 214 222 230
for 209 206 204
r 209 208 193
g 204 199 199
f 201 196 205
d 195 195 170
a 194 195 184
b 193 191 199
e 192 203 186
ic 180 180 177
n 176 176 180
was 176 176 161
o 175 175 169
l 171 167 174
h 167 173 178
w 166 166 171
am 161 161 164
el 159 156 158
ad 158 157 141
en 157 156 156
re 157 156 139
ac 155 149 143
you 153 153 148
st 151 151 165
are 150 150 148
with 149 149 138
sh 143 143 144
ar 143 143 151
inthe 143 143 142
ch 140 140 132
all 139 140 142
this 139 139 135
but 137 137 139
im 134 134 131
id 131 131 130
k 130 129 125
th 122 122 143
ap 120 120 106
le 120 128 130
un 118 118 127
ation 118 117

KeyError: 'ct'

In [6]:
# clean
%lprun -f test test()

Computing in parallel
Normalize and jobs data...
... computed in 10.095965147018433 secs.

Build alphabet...
... computed in 0.013791084289550781 secs.

Alphabet Size: 121
Special Tokens Size: 5
Terms to compute: 1874

Enter loop



Compute freq...
... computed in 3.718216896057129 secs.



Timer unit: 1e-06 s

Total time: 225.717 s
File: <ipython-input-4-d971a3e1e977>
Function: test at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def test(
     2                                               vocab_size = vocab_size,
     3                                               corpus_length = corpus_length,
     4                                           
     5                                               self = self,
     6                                               corpus = corpus,
     7                                           
     8                                               special_tokens = special_tokens,
     9                                               save = save,
    10                                               save_step = save_step,
    11                                               progress_bar = progress_bar,
    12                                               resume_merg

In [17]:
# max
%lprun -f test test()

Computing in parallel
Normalize and jobs data...
... computed in 11.31916093826294 secs.

Build alphabet...
... computed in 0.01871466636657715 secs.

Alphabet Size: 121
Special Tokens Size: 5
Terms to compute: 1874

Enter loop



Compute freq...
... computed in 4.060492992401123 secs.



Timer unit: 1e-06 s

Total time: 248.754 s
File: <ipython-input-16-3b7b167b1b1a>
Function: test at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def test(
     2                                               vocab_size = vocab_size,
     3                                               corpus_length = corpus_length,
     4                                           
     5                                               self = self,
     6                                               corpus = corpus,
     7                                           
     8                                               special_tokens = special_tokens,
     9                                               save = save,
    10                                               save_step = save_step,
    11                                               truncate_best_size = truncate_best_size,
    12                                             

In [14]:
# most_common
%lprun -f test test()

Computing in parallel
Normalize and jobs data...
... computed in 10.37221622467041 secs.

Build alphabet...
... computed in 0.013827085494995117 secs.

Alphabet Size: 121
Special Tokens Size: 5
Terms to compute: 3874

Enter loop



*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-06 s

Total time: 268.574 s
File: <ipython-input-13-f8e5f9cef15b>
Function: test at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def test(
     2                                               vocab_size = vocab_size,
     3                                               corpus_length = corpus_length,
     4                                           
     5                                               self = self,
     6                                               corpus = corpus,
     7                                           
     8                                               special_tokens = special_tokens,
     9                                               save = save,
    10                                               save_step = save_step,
    11                                               truncate_best_size = truncate_best_size,
    12                                             